![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LinearRegression 
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
 

In [2]:
# Load the dataset
movie_rentals = pd.read_csv('rental_info.csv') 
print(movie_rentals.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB
None


In [3]:
# Explore Data 
movie_rentals.head(10)

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
5,2005-05-29 16:51:44+00:00,2005-06-01 21:43:44+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
6,2005-06-17 19:42:42+00:00,2005-06-22 20:39:42+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
7,2005-07-09 18:23:46+00:00,2005-07-13 19:04:46+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
8,2005-07-27 13:16:28+00:00,2005-07-28 13:40:28+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
9,2005-08-21 13:53:52+00:00,2005-08-25 09:03:52+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [4]:
# Convert date columns to datetime format
movie_rentals['rental_date'] = pd.to_datetime(movie_rentals['rental_date']) 
movie_rentals['return_date'] = pd.to_datetime(movie_rentals['return_date'])

# Create rental length and rental length days column from rental and return dates
movie_rentals['rental_length'] = pd.to_datetime(
    movie_rentals["return_date"]) - pd.to_datetime(movie_rentals["rental_date"]) 
  
movie_rentals['rental_length_days'] = movie_rentals['rental_length'].dt.days 

movie_rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   rental_date         15861 non-null  datetime64[ns, UTC]
 1   return_date         15861 non-null  datetime64[ns, UTC]
 2   amount              15861 non-null  float64            
 3   release_year        15861 non-null  float64            
 4   rental_rate         15861 non-null  float64            
 5   length              15861 non-null  float64            
 6   replacement_cost    15861 non-null  float64            
 7   special_features    15861 non-null  object             
 8   NC-17               15861 non-null  int64              
 9   PG                  15861 non-null  int64              
 10  PG-13               15861 non-null  int64              
 11  R                   15861 non-null  int64              
 12  amount_2            15861 non-nu

In [5]:
# Determine the number of unique values in the 'special_features' column
movie_rentals['special_features'].value_counts()

special_features
{Trailers,Commentaries,"Behind the Scenes"}                     1308
{Trailers}                                                      1139
{Trailers,Commentaries}                                         1129
{Trailers,"Behind the Scenes"}                                  1122
{"Behind the Scenes"}                                           1108
{Commentaries,"Deleted Scenes","Behind the Scenes"}             1101
{Commentaries}                                                  1089
{Commentaries,"Behind the Scenes"}                              1078
{Trailers,"Deleted Scenes"}                                     1047
{"Deleted Scenes","Behind the Scenes"}                          1035
{"Deleted Scenes"}                                              1023
{Commentaries,"Deleted Scenes"}                                 1011
{Trailers,Commentaries,"Deleted Scenes","Behind the Scenes"}     983
{Trailers,Commentaries,"Deleted Scenes"}                         916
{Trailers,"Delete

In [6]:
# Create binary columns for special features
movie_rentals["deleted_scenes"] =  np.where(
    movie_rentals["special_features"].str.contains("Deleted Scenes"), 1,0) 
 
movie_rentals["behind_the_scenes"] = np.where(
    movie_rentals["special_features"].str.contains("Behind the Scenes"), 1,0) 
  
movie_rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   rental_date         15861 non-null  datetime64[ns, UTC]
 1   return_date         15861 non-null  datetime64[ns, UTC]
 2   amount              15861 non-null  float64            
 3   release_year        15861 non-null  float64            
 4   rental_rate         15861 non-null  float64            
 5   length              15861 non-null  float64            
 6   replacement_cost    15861 non-null  float64            
 7   special_features    15861 non-null  object             
 8   NC-17               15861 non-null  int64              
 9   PG                  15861 non-null  int64              
 10  PG-13               15861 non-null  int64              
 11  R                   15861 non-null  int64              
 12  amount_2            15861 non-nu

In [7]:
# Define the features and target variable
X = movie_rentals.drop(columns=['rental_date', 'return_date', 'rental_length', 'rental_length_days', 'special_features',], axis=1)
y = movie_rentals['rental_length_days']

print(X.head())

   amount  release_year  rental_rate  length  replacement_cost  NC-17  PG  \
0    2.99        2005.0         2.99   126.0             16.99      0   0   
1    2.99        2005.0         2.99   126.0             16.99      0   0   
2    2.99        2005.0         2.99   126.0             16.99      0   0   
3    2.99        2005.0         2.99   126.0             16.99      0   0   
4    2.99        2005.0         2.99   126.0             16.99      0   0   

   PG-13  R  amount_2  length_2  rental_rate_2  deleted_scenes  \
0      0  1    8.9401   15876.0         8.9401               0   
1      0  1    8.9401   15876.0         8.9401               0   
2      0  1    8.9401   15876.0         8.9401               0   
3      0  1    8.9401   15876.0         8.9401               0   
4      0  1    8.9401   15876.0         8.9401               0   

   behind_the_scenes  
0                  1  
1                  1  
2                  1  
3                  1  
4                  1  


In [8]:
# Perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [10]:
# Instantiate and fit a Lasso regression model and get the coefficients
lasso = Lasso(alpha=0.3, random_state=9)
lasso_coef = lasso.fit(X, y).coef_ 

print(lasso_coef)

[ 5.87567707e-01  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  4.32652862e-02  2.55377037e-06 -1.52154989e-01
 -0.00000000e+00  0.00000000e+00]


In [11]:
# Select the features with non-zero coefficients 
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

In [13]:
# Instantiate and fit a Linear Regression model using the selected features
lr = LinearRegression()
lr = lr.fit(X_lasso_train, y_train) 

# Predict on the test set and calculate the mean squared error
y_test_pred = lr.predict(X_lasso_test) 
mse_lr_lasso = mean_squared_error(y_test, y_test_pred) 

print(mse_lr_lasso)

4.812297241276236


In [15]:
# Define the parameter grid for Randomized Search
params = {'n_estimators': np.arange(1, 101, 1), 'max_depth':np.arange(1,11,1)} 

# Instantiate the Random Forest Regressor 
rf = RandomForestRegressor()  

# Instantiate the Randomized Search with cross-validation
rand_search = RandomizedSearchCV(rf, param_distributions=params, cv=5, random_state=5) 

# Fit the Randomized Search to the training data
rand_search.fit(X_train, y_train) 

# Get the best parameters from the Randomized Search
best_params = rand_search.best_params_  

print(best_params)

{'n_estimators': 99, 'max_depth': 10}


In [ ]:
# Instantiate the Random Forest Regressor with the best parameters
rf = RandomForestRegressor(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], random_state=9)

# Fit the Random Forest model to the training data
rf.fit(X_train, y_train)  

# Predict on the test set and calculate the mean squared error
rf_pred = rf.predict(X_test) 
mse_rf = mean_squared_error(y_test, rf_pred) 

print(mse_rf)

2.216615300433908


In [17]:
# Best model and mse 
best_model = rf
best_mse = mse_rf